In [ ]:
import os
import pickle
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import sklearn.metrics as metrics
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import ModelCheckpoint
from keras import models

In [ ]:
rm -rf ./untitled_project/

In [ ]:
!pip install imgaug==0.4.0
!pip install imgo
!pip install keras-tuner

In [ ]:
from imgo import uptools
from keras_tuner import HyperModel
from keras_tuner.tuners import BayesianOptimization

In [ ]:
class_names = ['amethyst_deceiver',
               'bolete',
               'chanterelle',
               'chicken_of_the_woods',
               'death_cap',
               'false_chanterelle',
               'false_morel',
               'fibrecap',
               'field_mushroom',
               'fly_agaric',
               'giant_puffball',
               'grey_oyster',
               'morel',
               'orange_peel',
               'roundhead',
               'saddle',
               'shaggy_inkcap',
               'stinkhorn',
               'waxcap',
               'yellow_stainer']


In [ ]:
%%time
ds = uptools.Image_Dataset("/floyd/input/ds_1", "h5", 256, normalize=True, manual_classes=class_names)

In [ ]:
ds.details(plot=True)

In [ ]:
class CNNHyperModel(HyperModel):
    def __init__(self,input_shape,class_no):
        self.input_shape = input_shape
        self.num_classes = class_no

    def build(self,hp):

        inputs = keras.Input(shape=self.input_shape)
        x = layers.Conv2D(32,(3,3),activation='relu',padding='same')(inputs)
        x = layers.MaxPooling2D((2,2))(x)
        x = layers.Conv2D(64,(3,3),activation='relu',padding='same')(x)
        x = layers.MaxPooling2D((2,2))(x)
        x = layers.Conv2D(128,(3,3),activation='relu',padding='same')(x)
        x = layers.MaxPooling2D((2,2))(x)
        x = layers.Conv2D(256,(3,3),activation='relu',padding='same')(x)
        x = layers.MaxPooling2D((2,2))(x)
        x = layers.Conv2D(512,(3,3),activation='relu',padding='same')(x)
        x = layers.MaxPooling2D((2,2))(x)
        x = layers.Flatten()(x)
        x = layers.Dense(1024, activation='relu')(x)
        x = layers.Dropout(rate=hp.Float('dropout_2',min_value=0.0,max_value=0.5,default=0.25,step=0.03,))(x)
        outputs = layers.Dense(self.num_classes, activation='softmax')(x)

        model = keras.Model(inputs=inputs, outputs=outputs)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        model.summary()

        return model
    
hypermodel = CNNHyperModel(ds.img_shape,ds.class_no)

In [ ]:
n_epochs = 8
s_batch = 32
m_trials = 10

In [ ]:
%%time
tuner = BayesianOptimization(hypermodel,
                             max_trials=m_trials,
                             objective='val_accuracy')

tuner.search(ds.X_train,
             ds.y_train,
             epochs=n_epochs,
             validation_data=(ds.X_val,ds.y_val),
             batch_size=s_batch)

In [ ]:
tuner.results_summary()
loss, accuracy = best_model.evaluate(ds.X_val, ds.y_val)

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
(val_loss, val_acc) = best_model.evaluate(ds.X_val,ds.y_val,verbose=1)
print('---')
print(f'Val Loss: {val_loss}')
print(f'Val Accuracy: {val_acc}')